## RNN 개요

RNN (순환 신경망) : 시퀀스 모델로 입력과 출력을 시퀀스 단위로 처리.
    

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import SimpleRNN
#tf.enable_eager_execution()

In [2]:
# 일부만 할당된 GPU 메모리를 실행하는 동안 필요한만큼 늘릴 수 있도록 설정함
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
train_X = [[0.1,4.2,1.5,1.1,2.8],[1.0,3.1,2.5,0.7,1.1],[0.3,2.1,1.5,2.1,0.1],[2.2,1.4,0.5,0.9,1.1]]
print(np.shape(train_X))

(4, 5)


In [4]:
# input_length와 inpue dim의 정보만 가지고 있는 것이기 때문에 batch_size에 대한 dim으
train_X = [[[0.1,4.2,1.5,1.1,2.8],[1.0,3.1,2.5,0.7,1.1],[0.3,2.1,1.5,2.1,0.1],[2.2,1.4,0.5,0.9,1.1]]]
train_X=np.array(train_X, dtype=np.float32)
print(train_X.shape)

(1, 4, 5)


In [5]:
SimpleRNN?

In [6]:
rnn = SimpleRNN(3, return_sequences =True, return_state=True)
#SimpleRNN(units) : 하나의 셀에서 몇개의

hidden_states, last_states = rnn(train_X)

In [7]:
print('train_X : {}, shape : {}\n'.format(train_X, train_X.shape)) #입력으로 사용한 훈련 데이터
print('hidden states : {}, shape : {}\n'.format(hidden_states, hidden_states.shape)) #모든 time_step의 은닉 상태 : (x1 * w1)의 상태
print('last hidden state : {}, shape : {}'.format(last_states, last_states.shape)) # 마지막 은닉 상태

train_X : [[[0.1 4.2 1.5 1.1 2.8]
  [1.  3.1 2.5 0.7 1.1]
  [0.3 2.1 1.5 2.1 0.1]
  [2.2 1.4 0.5 0.9 1.1]]], shape : (1, 4, 5)

hidden states : [[[ 0.931975    0.6314502  -0.9998573 ]
  [ 0.99927753  0.71003866 -0.9992818 ]
  [ 0.9455069   0.71973026 -0.99318105]
  [ 0.9902862   0.96816707 -0.9247837 ]]], shape : (1, 4, 3)

last hidden state : [[ 0.9902862   0.96816707 -0.9247837 ]], shape : (1, 3)


여기서부터는 케라스로 tf.enable_eager_execution() 때문에 다시 restart하고 해야함.

In [1]:
text = "나랑 점심 먹으러 갈래 메뉴는 햄버거 점심 메뉴 좋지"

In [2]:
from keras_preprocessing.text import Tokenizer

t= Tokenizer()
t.fit_on_texts([text])

#[0]을 해주지 않으면 [[contents]]처럼 리스트 안에 리스트 형태로 저장이 되버림
#[0]을 해주면 하나의 리스트로 저장됨 [content]

In [3]:
#케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하고
# 케라스 원-핫 인코딩에서 배열의 인덱스는 0부터 시작하기 때문에
#배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1을 선언해줌

vocab_size=len(t.word_index)+1
print('단어 집합의 크기 : %d' %vocab_size)

단어 집합의 크기 : 9


In [4]:
# 각 단어의 빈도수에 따라서 부여된 인덱스 확인
print( t.word_index)

{'점심': 1, '나랑': 2, '먹으러': 3, '갈래': 4, '메뉴는': 5, '햄버거': 6, '메뉴': 7, '좋지': 8}


In [5]:
# 무장에서 두개 단어씩 묶어주기 : 여기서 문장의 Bigram을 추출한다고 봐도 무방함

In [6]:
sequences = list()
for c in range(1, len(encoded)):
    sequence = encoded[c-1:c+1]
    sequences.append(sequence)
print('단어 묶음의 개수: %d' % len(sequences))

단어 묶음의 개수: 8


In [7]:
print(sequences)

[[2, 1], [1, 3], [3, 4], [4, 5], [5, 6], [6, 1], [1, 7], [7, 8]]


In [8]:
import numpy as np
X, y = zip(*sequences)
X=np.array(X)
y=np.array(y)

In [9]:
y

array([1, 3, 4, 5, 6, 1, 7, 8])

In [10]:
from keras.utils import to_categorical

y = to_categorical(y, num_classes= vocab_size) # 원핫 인코딩한 개수 9개vocab_size=len(t.word_index)+1
print(y)

Using TensorFlow backend.


[[0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [11]:
from keras.layers import Embedding, Dense, SimpleRNN
from keras.models import Sequential

model=Sequential()
model.add(Embedding(vocab_size, 9, input_length=1))
#단어 집합의 크기는 9, 임베딩 벡터의 크기는 9, 각 sample의 길이는 단어 한 개이므로 길이는 1.
model.add(SimpleRNN(9))
#RNN의 결과값으로 나오는 벡터의 차원 또한 9로 한다. 더 크게 해주어도 상관은 없음. vocab_size보다 작아지면 생각해봐야함.
model.add(Dense(vocab_size, activation='softmax'))
#출력층을 지나서 나오는 벡터의 크기도 9로 한다.

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y, epochs=500, verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
 - 2s - loss: 2.2068 - acc: 0.0000e+00
Epoch 2/500
 - 0s - loss: 2.2032 - acc: 0.0000e+00
Epoch 3/500
 - 0s - loss: 2.1997 - acc: 0.0000e+00
Epoch 4/500
 - 0s - loss: 2.1962 - acc: 0.0000e+00
Epoch 5/500
 - 0s - loss: 2.1927 - acc: 0.0000e+00
Epoch 6/500
 - 0s - loss: 2.1892 - acc: 0.0000e+00
Epoch 7/500
 - 0s - loss: 2.1857 - acc: 0.0000e+00
Epoch 8/500
 - 0s - loss: 2.1823 - acc: 0.1250
Epoch 9/500
 - 0s - loss: 2.1788 - acc: 0.2500
Epoch 10/500
 - 0s - loss: 2.1754 - acc: 0.2500
Epoch 11/500
 - 0s - loss: 2.1719 - acc: 0.3750
Epoch 12/500
 - 0s - loss: 2.1685 - acc: 0.3750
Epoch 13/500
 - 0s - loss: 2.1650 - acc: 0.3750
Epoch 14/500
 - 0s - loss: 2.1615 - acc: 0.3750
Epoch 15/500
 - 0s - loss: 2.1581 - acc: 0.3750
Epoch 16/500
 - 0s - loss: 2.1546 - acc: 0.3750
Epoch 17/500
 - 0s - loss: 2.1511 - acc: 0.3750
Epoch 18/500
 - 0s - loss: 2.1476 - acc: 0.3750
Epoch 19/500
 - 0s - loss: 2.1440 - acc: 0.3750
Epoch 20/500
 - 0s - 

Epoch 165/500
 - 0s - loss: 1.1661 - acc: 0.7500
Epoch 166/500
 - 0s - loss: 1.1590 - acc: 0.7500
Epoch 167/500
 - 0s - loss: 1.1520 - acc: 0.7500
Epoch 168/500
 - 0s - loss: 1.1450 - acc: 0.7500
Epoch 169/500
 - 0s - loss: 1.1380 - acc: 0.7500
Epoch 170/500
 - 0s - loss: 1.1311 - acc: 0.7500
Epoch 171/500
 - 0s - loss: 1.1242 - acc: 0.7500
Epoch 172/500
 - 0s - loss: 1.1173 - acc: 0.7500
Epoch 173/500
 - 0s - loss: 1.1105 - acc: 0.7500
Epoch 174/500
 - 0s - loss: 1.1037 - acc: 0.7500
Epoch 175/500
 - 0s - loss: 1.0969 - acc: 0.7500
Epoch 176/500
 - 0s - loss: 1.0901 - acc: 0.7500
Epoch 177/500
 - 0s - loss: 1.0834 - acc: 0.7500
Epoch 178/500
 - 0s - loss: 1.0767 - acc: 0.7500
Epoch 179/500
 - 0s - loss: 1.0701 - acc: 0.7500
Epoch 180/500
 - 0s - loss: 1.0634 - acc: 0.7500
Epoch 181/500
 - 0s - loss: 1.0568 - acc: 0.7500
Epoch 182/500
 - 0s - loss: 1.0503 - acc: 0.7500
Epoch 183/500
 - 0s - loss: 1.0437 - acc: 0.7500
Epoch 184/500
 - 0s - loss: 1.0372 - acc: 0.7500
Epoch 185/500
 - 0s 

Epoch 333/500
 - 0s - loss: 0.4052 - acc: 0.8750
Epoch 334/500
 - 0s - loss: 0.4033 - acc: 0.8750
Epoch 335/500
 - 0s - loss: 0.4015 - acc: 0.8750
Epoch 336/500
 - 0s - loss: 0.3997 - acc: 0.8750
Epoch 337/500
 - 0s - loss: 0.3979 - acc: 0.8750
Epoch 338/500
 - 0s - loss: 0.3961 - acc: 0.8750
Epoch 339/500
 - 0s - loss: 0.3944 - acc: 0.8750
Epoch 340/500
 - 0s - loss: 0.3926 - acc: 0.8750
Epoch 341/500
 - 0s - loss: 0.3909 - acc: 0.8750
Epoch 342/500
 - 0s - loss: 0.3892 - acc: 0.8750
Epoch 343/500
 - 0s - loss: 0.3876 - acc: 0.8750
Epoch 344/500
 - 0s - loss: 0.3859 - acc: 0.8750
Epoch 345/500
 - 0s - loss: 0.3843 - acc: 0.8750
Epoch 346/500
 - 0s - loss: 0.3827 - acc: 0.8750
Epoch 347/500
 - 0s - loss: 0.3811 - acc: 0.8750
Epoch 348/500
 - 0s - loss: 0.3795 - acc: 0.8750
Epoch 349/500
 - 0s - loss: 0.3779 - acc: 0.8750
Epoch 350/500
 - 0s - loss: 0.3764 - acc: 0.8750
Epoch 351/500
 - 0s - loss: 0.3748 - acc: 0.8750
Epoch 352/500
 - 0s - loss: 0.3733 - acc: 0.8750
Epoch 353/500
 - 0s 

In [13]:
model.predict(X)

array([[1.4815362e-03, 9.5107222e-01, 1.6035764e-03, 1.8427371e-03,
        4.5146826e-03, 1.9242615e-02, 1.9892729e-03, 1.2654323e-02,
        5.5991299e-03],
       [5.4348498e-03, 1.0263404e-02, 6.4641004e-03, 4.7101909e-01,
        6.0516276e-04, 1.7061604e-02, 2.5241606e-02, 4.6188375e-01,
        2.0264499e-03],
       [1.7265070e-02, 7.1216477e-03, 1.5110824e-02, 1.2657852e-03,
        9.0148973e-01, 7.4216533e-03, 3.4885330e-03, 1.4468402e-03,
        4.5389876e-02],
       [3.9107678e-03, 2.9963385e-02, 4.6100039e-03, 4.1733216e-03,
        3.3667509e-03, 9.3124020e-01, 1.9523232e-04, 2.2481531e-02,
        5.8877486e-05],
       [3.7328932e-03, 1.0261605e-03, 5.1826267e-03, 2.9493028e-02,
        3.2641962e-03, 8.5462460e-05, 8.7846243e-01, 6.8890760e-03,
        7.1864039e-02],
       [1.4497860e-03, 9.5088404e-01, 1.5691913e-03, 2.1171782e-03,
        4.1852929e-03, 1.8672161e-02, 2.0380998e-03, 1.3853672e-02,
        5.2305330e-03],
       [5.4348498e-03, 1.0263404e-02, 6.

In [14]:
print(t.word_index.items())

dict_items([('점심', 1), ('나랑', 2), ('먹으러', 3), ('갈래', 4), ('메뉴는', 5), ('햄버거', 6), ('메뉴', 7), ('좋지', 8)])


In [15]:
def predict_next_word(model, t, current_word) :
    encoded = t.texts_to_sequences([current_word])[0]
    encoded = np.array(encoded)
    result = model.predict_classes(encoded, verbose=0)
    for word, index in t.word_index.items() :
        if index == result :
            return word

In [16]:
print(predict_next_word(model, t, '먹으러'))

갈래


In [17]:
def sentence_generation(model, t, current_word, n) :
    init_word = current_word
    sentence = ''
    for _ in range(n) : #n번 반복
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = np.array(encoded)
        result = model.predict_classes(encoded, verbose = 0)
        for word, index in t.word_index.items() :
            if index == result :
                break
        current_word = word
        sentence = sentence + ' ' + word
    sentence = init_word + sentence
    return sentence

In [25]:
print(sentence_generation(model, t, '메뉴', 5))

메뉴 좋지 점심 먹으러 갈래 메뉴는


# 문맥을 고려한 text 예상

In [1]:
text='''경마장에 있는 말이 뛰고 있다.\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n'''

In [2]:
from keras_preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts([text])
encoded = t.texts_to_sequences([text])[0]

In [3]:
vocab_size = len(t.word_index)+1
#토크나이저는 1부터 원-핫 인코딩은 0부터 시작해서
print('단어 집합의 크기 :', vocab_size)
print(t.word_index)

단어 집합의 크기 : 12
{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [4]:
sequences=list()
for line in text.split('\n'): #\n을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0] #[0] text_to_sequences는 [0]을 해줘야 리스트안의 리스트로 출력되지 않음.
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
print('훈련 데이터의 개수 : %d' % len(sequences))

훈련 데이터의 개수 : 11


In [5]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [6]:
print(max(len(l) for l in sequences)) # sequences의 모든 데이터에서 길이를 구해서 그 중 max를 출력

6


In [7]:
from keras.preprocessing.sequence import pad_sequences

sequences = pad_sequences(sequences, maxlen=6, padding='pre')

print(sequences)

Using TensorFlow backend.


[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [8]:
import numpy as np
sequences = np.array(sequences) # array형태로 만들어줌
X = sequences[:,:-1] # 마지막 열만 뺴고 X
y = sequences[:,-1] #마지막 열은 y로

print(X,'\n\n',y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]] 

 [ 3  1  4  5  1  7  1  9 10  1 11]


In [9]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes = vocab_size)

In [23]:
print(y)
print(y.reshape())

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
(11, 12)


In [24]:
from keras.layers import Embedding, Dense, SimpleRNN
from keras.models import Sequential

model=Sequential()
model.add(Embedding(vocab_size, 10, input_length=5)) # X 길이는 y를 제거해서 5가 됨
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y,epochs=200, verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
 - 2s - loss: 2.4880 - acc: 0.2727
Epoch 2/200
 - 0s - loss: 2.4736 - acc: 0.2727
Epoch 3/200
 - 0s - loss: 2.4589 - acc: 0.1818
Epoch 4/200
 - 0s - loss: 2.4439 - acc: 0.3636
Epoch 5/200
 - 0s - loss: 2.4285 - acc: 0.3636
Epoch 6/200
 - 0s - loss: 2.4127 - acc: 0.3636
Epoch 7/200
 - 0s - loss: 2.3965 - acc: 0.3636
Epoch 8/200
 - 0s - loss: 2.3796 - acc: 0.3636
Epoch 9/200
 - 0s - loss: 2.3621 - acc: 0.3636
Epoch 10/200
 - 0s - loss: 2.3440 - acc: 0.3636
Epoch 11/200
 - 0s - loss: 2.3251 - acc: 0.3636
Epoch 12/200
 - 0s - loss: 2.3056 - acc: 0.3636
Epoch 13/200
 - 0s - loss: 2.2852 - acc: 0.3636
Epoch 14/200
 - 0s - loss: 2.2641 - acc: 0.3636
Epoch 15/200
 - 0s - loss: 2.2423 - acc: 0.3636
Epoch 16/200
 - 0s - loss: 2.2198 - acc: 0.3636
Epoch 17/200
 - 0s - loss: 2.1966 - acc: 0.3636
Epoch 18/200
 - 0s - loss: 2.1730 - acc: 0.3636
Epoch 19/200
 - 0s - loss: 2.1491 - acc: 0.3636
Epoch 20/200
 - 0s - loss: 2.1250 - acc: 0.3636
E

Epoch 165/200
 - 0s - loss: 0.1931 - acc: 1.0000
Epoch 166/200
 - 0s - loss: 0.1894 - acc: 1.0000
Epoch 167/200
 - 0s - loss: 0.1858 - acc: 1.0000
Epoch 168/200
 - 0s - loss: 0.1822 - acc: 1.0000
Epoch 169/200
 - 0s - loss: 0.1788 - acc: 1.0000
Epoch 170/200
 - 0s - loss: 0.1754 - acc: 1.0000
Epoch 171/200
 - 0s - loss: 0.1720 - acc: 1.0000
Epoch 172/200
 - 0s - loss: 0.1688 - acc: 1.0000
Epoch 173/200
 - 0s - loss: 0.1656 - acc: 1.0000
Epoch 174/200
 - 0s - loss: 0.1625 - acc: 1.0000
Epoch 175/200
 - 0s - loss: 0.1594 - acc: 1.0000
Epoch 176/200
 - 0s - loss: 0.1565 - acc: 1.0000
Epoch 177/200
 - 0s - loss: 0.1535 - acc: 1.0000
Epoch 178/200
 - 0s - loss: 0.1507 - acc: 1.0000
Epoch 179/200
 - 0s - loss: 0.1479 - acc: 1.0000
Epoch 180/200
 - 0s - loss: 0.1452 - acc: 1.0000
Epoch 181/200
 - 0s - loss: 0.1425 - acc: 1.0000
Epoch 182/200
 - 0s - loss: 0.1399 - acc: 1.0000
Epoch 183/200
 - 0s - loss: 0.1374 - acc: 1.0000
Epoch 184/200
 - 0s - loss: 0.1349 - acc: 1.0000
Epoch 185/200
 - 0s 

# 문맥을 고려한 text 예측 모델2

In [29]:
import pandas as pd
df=pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [30]:
print(df.columns)
print('열의 개수 : ', len(df.columns))

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')
열의 개수 :  15


In [31]:
df['headline'].isnull().values.any()

False

In [32]:
headline = [] #리스트 선언
headline.extend(list(df.headline.values))#헤드라인의 값들을 리스트로 저장
headline[:5] #상위 5개 출력

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [33]:
len(headline) # 현재 데이터의 개수

1324

In [34]:
headline = [n for n in headline if n != "Unknown"]

In [35]:
len(headline) # 제거 후 데이터의 개수

1214

In [36]:
headline[:5] # unknown 이 제거된 것을 확인

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [38]:
from string import punctuation
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii", "ignore") #원래 지금은 아스키 코드로 되어있는데 보이긴 하지만 , decode 시켜서 encode 시켜서 punctuation이 읽을 수 있게 해줌
    return ''.join(c for c in s if c not in punctuation).lower() #구두점 제거와 동시에 소문자화

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [42]:
from keras_preprocessing.text import Tokenizer
t=Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) +1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [43]:
sequences = list()

for line in text: #1214 개의 데이터에 대해서 1개씩 가져온다. 
    encoded = t.texts_to_sequences([line])[0] # 각 데이터에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence =encoded[:i+1]
        sequences.append(sequence)
        
sequences[:11] #11개의 데이터 출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [44]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

index_to_word[582]

'offer'

In [45]:
max_len = max(len(l) for l in sequences)
print(max_len)

24


In [46]:
from keras.preprocessing.sequence import pad_sequences
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]
